In [6]:
import yfinance as yf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


In [7]:
# BTC-USD için veri çekme
data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')
close_prices = data['Close'].values.reshape(-1, 1)

# Veriyi normalize etme
scaler = MinMaxScaler(feature_range=(0, 1))
close_prices_scaled = scaler.fit_transform(close_prices)

# Veriyi Seq2Seq modeline uygun hale getirme
def create_dataset(dataset, n_past=60, n_future=1):
    X, y = [], []
    for i in range(n_past, len(dataset) - n_future +1):
        X.append(dataset[i - n_past:i, 0])
        y.append(dataset[i:i + n_future, 0])
    return np.array(X), np.array(y)

n_past = 60
n_future = 1
X, y = create_dataset(close_prices_scaled, n_past, n_future)
X = X.reshape((X.shape[0], X.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))


[*********************100%%**********************]  1 of 1 completed


In [8]:
# Encoder
encoder_inputs = Input(shape=(n_past, 1))
encoder_l1 = LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]

# Decoder
decoder_inputs = Input(shape=(n_future, 1))
decoder_l1 = LSTM(100, return_sequences=True, return_state=True)
decoder_outputs1 = decoder_l1(decoder_inputs, initial_state=encoder_states1)

decoder_dense = Dense(1, activation='linear')
decoder_outputs = decoder_dense(decoder_outputs1[0])

# Seq2Seq Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='mean_squared_error')


In [9]:
decoder_input_data = np.zeros_like(X)
decoder_input_data[:,1:,0] = y[:,:-1,0]
decoder_target_data = y

model.fit([X, decoder_input_data], decoder_target_data, batch_size=32, epochs=10, validation_split=0.2)


ValueError: could not broadcast input array from shape (1036,0) into shape (1036,59)